# AutoGraph

In [1]:
import os

import tensorflow as tf
import numpy as np

os.environ["CUDA_VISIBLE_DEVICES"] = "3"
tf.__version__

'2.1.0'

有三种计算图的构建方式：静态计算图，动态计算图，以及Autograph。

TensorFlow 2.0主要使用的是动态计算图和Autograph。

动态计算图易于调试，编码效率较高，但执行效率偏低。

静态计算图执行效率很高，但较难调试。

而Autograph机制可以将动态图转换成静态计算图，兼收执行效率和编码效率之利。

## AutoGraph的机制原理

In [2]:
# 当执行一个 被@tf.function装饰 的函数时，会发什么？

@tf.function(autograph=True)
def myadd(a, b):
    for i in tf.range(3):
        tf.print(i)
    c = a + b
    print("tracing")
    print(c)
    return c

In [3]:
# 执行myadd调用

a = tf.constant("hello")
b = tf.constant("world")

print(f"a shape={a.shape}")
print(f"a data={a}")
print()

print(f"b shape={b.shape}")
print(f"b data={b}")
print()

c = myadd(a, b)
print()
print(f"c shape={c.shape}")
print(f"c data={c}")
print()

a shape=()
a data=b'hello'

b shape=()
b data=b'world'

tracing
Tensor("add:0", shape=(), dtype=string)
0
1
2

c shape=()
c data=b'helloworld'



### 执行过程

被 **@tf.function** 装饰的函数的执行过程：
1. 创建一个静态计算图；
2. 执行一遍**函数体**(function body)中的Python代码，确定各个变量的**Tensor类型**，并根据执行顺序将算子添加到计算图中。 
    > 在这个过程中，如果开启了autograph=True(默认开启),会将Python控制流转换成TensorFlow图内控制流。 
    > - 主要是将 **if语句** 转换成 **tf.cond算子表达**；
    > - 将 **while和for循环语句** 转换成 **tf.while_loop算子表达**；
    > - 并在 *必要* 的时候添加 **tf.control_dependencies** 指定 **执行顺序依赖关系**；
    > 
    > (类似 执行下面的 TensorFlow-1 代码)
3. 执行静态计算图；

``` Python
# 第1步
g = tf.Graph()

# 第2步
with g.as_default():
    a = tf.placeholder(shape=[], dtype=tf.string)
    b = tf.placeholder(shape=[], dtype=tf.string)
    cond = lambda i: i<tf.constant(3)
    def body(i):
        tf.print(i)
        return(i + 1)
    loop = tf.while_loop(cond, body, loop_vars=[0])
    loop
    with tf.control_dependencies(loop):
        c = tf.strings.join([a, b])
    print("tracing")

# 第3步
with tf.Session(graph=g) as sess:
    d_feed={
        a: tf.constant("hello"),
        b: tf.constant("world")
    }
    sess.run(c, feed_dict=d_feed)
```

In [4]:
# 使用 【相同的】 【输入参数类型】 调用被@tf.function装饰的函数，不会在创建新的计算图，而是直接计算结果
c = myadd(a, b)  # 不会输出 “tracing”
print()
print(f"c shape={c.shape}")
print(f"c data={c}")
print()

0
1
2

c shape=()
c data=b'helloworld'



In [5]:
# 使用 【不同的】 【输入参数类型】 再次调用被@tf.function装饰的函数，会重新创建新的计算图，然后在进行计算
c = myadd(tf.constant(10), tf.constant(20))  # 会输出 “tracing”
print()
print(f"c shape={c.shape}")
print(f"c data={c}")
print()

tracing
Tensor("add:0", shape=(), dtype=int32)
0
1
2

c shape=()
c data=30



In [6]:
# 使用 【非Tensor类型】 的输入参数 调用被@tf.function装饰的函数，每次都会重新创建新的计算图，然后在进行计算
c = myadd("hello", "world")  # 会输出 “tracing”
print()
print(f"c shape={c.shape}")
print(f"c data={c}")
print()
print("===" * 30)
print()
c = myadd("good", "morning")  # 会输出 “tracing”
print()
print(f"c shape={c.shape}")
print(f"c data={c}")
print()

tracing
helloworld
0
1
2

c shape=()
c data=b'helloworld'


tracing
goodmorning
0
1
2

c shape=()
c data=b'goodmorning'



### AutoGraph编码规范总结

1. 被@tf.function修饰的函数应尽可能使用TensorFlow中的函数而不是Python中的其他函数。例如使用tf.print而不是print，使用tf.range而不是range，使用tf.constant(True)而不是True.

2. 避免在@tf.function修饰的函数内部定义tf.Variable.

3. 被@tf.function修饰的函数不可修改该函数外部的Python列表或字典等数据结构变量。

In [7]:
@tf.function
def np_random():
    a = np.random.randn(3, 3)
    tf.print(a)

@tf.function
def tf_random():
    a = tf.random.normal((3, 3))
    tf.print(a)

In [8]:
#np_random每次执行都是一样的结果。
np_random()
np_random()

array([[ 1.43561844, -0.25512613, -1.01469098],
       [ 0.57805868,  0.19421299, -0.16486978],
       [ 1.26378418,  0.49975596, -0.95719563]])
array([[ 1.43561844, -0.25512613, -1.01469098],
       [ 0.57805868,  0.19421299, -0.16486978],
       [ 1.26378418,  0.49975596, -0.95719563]])


In [9]:
#tf_random每次执行都会有重新生成随机数。
tf_random()
tf_random()

[[0.685071588 1.08211172 -0.112555772]
 [0.250782341 0.46712181 1.44548643]
 [0.564745486 0.192541301 -0.354620546]]
[[1.52672839 -0.733891308 0.885099053]
 [-1.95072043 0.810730398 1.18380439]
 [0.874663532 0.139958337 -1.71984434]]


In [10]:
# 避免在@tf.function修饰的函数内部定义tf.Variable.

x = tf.Variable(1.0, dtype=tf.float32)

@tf.function
def outer_var():
    x.assign_add(1.0)
    tf.print(x)
    return(x)

outer_var()
outer_var()

2
3


<tf.Tensor: shape=(), dtype=float32, numpy=3.0>

In [11]:
@tf.function
def inner_var():
    x = tf.Variable(1.0, dtype = tf.float32)
    x.assign_add(1.0)
    tf.print(x)
    return(x)

#执行将报错
# inner_var()
#inner_var()

In [12]:
tensor_list = []

#@tf.function #加上这一行切换成Autograph结果将不符合预期！！！
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list

append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)

[<tf.Tensor: shape=(), dtype=float32, numpy=5.0>, <tf.Tensor: shape=(), dtype=float32, numpy=6.0>]


In [13]:
tensor_list = []

@tf.function # 加上这一行切换成Autograph结果将不符合预期！！！
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list


append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)

[<tf.Tensor 'x:0' shape=() dtype=float32>]


In [14]:
a = np.random.randint(0, high=2, size=(4, 4))
a

array([[0, 1, 0, 1],
       [0, 0, 0, 0],
       [1, 1, 0, 0],
       [0, 0, 0, 0]])